# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [217]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

In [116]:
df = pd.read_csv('homepage_actions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [114]:
# df['id'].value_counts()
# df = df.drop_duplicates(subset=['id'])
# df

#Dataframes of actions
view = df.loc[df['action'] == 'view']
click = df.loc[df['action'] == 'click']

#Dataframes of experiment vs control
exp = df.loc[df['group'] == 'experiment']
ctrl = df.loc[df['group'] == 'control']

print(exp['action'].value_counts())
print(ctrl['action'].value_counts())

# Making groups to aggregate on IDs
group = df.groupby(['id', 'action'])
group.first()

view     2996
click     928
Name: action, dtype: int64
view     3332
click     932
Name: action, dtype: int64


timestamp       group
id     action                                        
182988 view    2016-11-26 14:38:47.682931  experiment
182994 click   2016-12-14 13:20:29.564080     control
       view    2016-12-14 13:19:00.940677     control
183089 view    2016-09-29 23:04:42.722325     control
183136 view    2016-12-27 10:13:58.808014  experiment
...                                   ...         ...
937108 view    2017-01-07 11:50:17.313666     control
937139 click   2016-12-09 07:06:05.219267  experiment
       view    2016-12-09 07:05:04.965876  experiment
937217 click   2016-11-14 08:01:25.774204     control
       view    2016-11-14 08:00:01.730193     control

[8188 rows x 2 columns]

In [115]:
# Looking at groups with one entry
views = df.groupby('id').filter(lambda x: len(x) == 1)

# Checking for clicks without views
views['action'].value_counts()

#Looks good :-)

view    4468
Name: action, dtype: int64

In [163]:
# Checking for overlap:
experiment_id = set(df[df.group=='experiment']['id'].unique())
control_id = set(df[df.group=='control']['id'].unique())
len(experiment_id & control_id)

# No overlap!

0

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [246]:
alpha = 0.05

# From our previous grouping on id, we can look at ids with two actions
clicks = df.groupby('id').filter(lambda x: len(x) == 2)
# Removing view entries from click ids
clicks = clicks[clicks.action=='click']

# Looking at those ids who only view
only_view = df.groupby('id').filter(lambda x: len(x) == 1)

#combining these datasets to make a set without duplicates
nodupes = clicks.append(only_view)

nodupes['clickthrough'] = 0
nodupes['clickthrough'].loc[nodupes['action'] =='click'] = 1

# #Splitting into control and experiment dataframes
control_group = nodupes[(nodupes.group == 'control')]
experiment_group = nodupes[(nodupes.group == 'experiment')]

# Grabbing p-value with ttest
p_value = stats.ttest_ind(control_group['clickthrough'], experiment_group['clickthrough'], equal_var=False)

# Dividing by two, for a one-sided test
p_value = p_value[1] / 2

print(f'The p-value between the control and experiment is {round(p_value, 4)}')

The p-value between the control and experiment is 0.0045


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [248]:
# Grabbing mean from clickthrough group
con_ctr = control_group.clickthrough.mean()

# use this mean as a multiplier on the length of the experiment group
print(con_ctr * len(experiment_group))

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [257]:
# Calculating the standard deviation:
standdev = np.sqrt(len(experiment_group) * con_ctr * (1-con_ctr))

z_score = (experiment_group.clickthrough.sum() - con_ctr * len(experiment_group))/standdev
z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [259]:
p_value_2 = stats.norm.sf(z_score)
p_value_2

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **It doesn't change the result of the study, but it is a lower value. There may be instances where
this sort of check would change the outcome of a study.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.